# अंश ३: उन्नत दूरस्थ निष्पादन उपकरण (Advanced Remote Execution Tools)

अंतिम खंड में हमने फेडरेटेड लर्निंग (Federated Learning) का उपयोग करके एक खिलौना मॉडल का प्रशिक्षण लिया। हमने अपने मॉडल पर .send() और .get() कॉल करके, इसे प्रशिक्षण डेटा के स्थान पर भेजकर, इसे अपडेट करके, और फिर इसे वापस लाया। हालांकि, उदाहरण के अंत में हमने महसूस किया कि हमें लोगों की गोपनीयता की रक्षा के लिए थोड़ा और आगे जाने की आवश्यकता है। अर्थात्, हम .get() कॉल करने से पहले ग्रेडिएंट को औसत (average gradient) करना चाहते हैं। इस तरह, हम कभी किसी की सटीक ग्रेडिएंट (gradient) नहीं देखेंगे (इस प्रकार उनकी गोपनीयता की रक्षा करना बेहतर है!!!)

लेकिन, ऐसा करने के लिए, हमें कुछ और खंड चाहिए:

- एक Tensor को सीधे किसी अन्य Worker को भेजने के लिए एक Pointer का उपयोग करें

और इसके अलावा, जब हम यहां हैं, तो हम कुछ और उन्नत tensor ऑपरेशंस के बारे में भी जानने वाले हैं, जो भविष्य में इस उदाहरण और कुछ अन्य चीज़ों में हमारी मदद करेंगे!

लेखक:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

अनुवादक:
- Arkadip Bhattacharya - Github: [@darkmatter18](https://github.com/darkmatter18)

In [ ]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

# अनुभाग 3.1 - Pointers से Pointers

जैसा कि आप जानते हैं, PointerTensor वस्तुएं सामान्य Tensor की तरह ही होती हैं। वास्तव में, वह इतनी Tensor जैसे होते हैं कि हम भी Pointers से Pointers कर सकते हैं। चलो पता करते हैं!

In [ ]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [ ]:
# this is a local tensor
x = torch.tensor([1,2,3,4])
x

In [ ]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

In [ ]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

### क्या हुआ?

इसलिए, पिछले उदाहरण में, हमने `x` नामक एक Tensor बनाया और इसे Bob को भेजें, और हमारे स्थानीय मशीन (` x_ptr`) पर एक पॉइंटर बनाएं।

फिर, हमने `x_ptr.send (alice)` को कॉल किया, जिसने Alice को Pointer भेजा। 

ध्यान दें, यह डेटा स्थानांतरित नहीं किया था! इसके बजाय, यह Pointer को डेटा में ले गया !!

In [ ]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

In [ ]:
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
alice._objects

In [ ]:
# and we can use .get() to get x_ptr back from Alice

x_ptr = pointer_to_x_ptr.get()
x_ptr

In [ ]:
# and then we can use x_ptr to get x back from Bob!

x = x_ptr.get()
x

### Pointer -> Pointer -> Data Object पर अंकगणित (Arithmetic)

और सामान्य Pointers की तरह, हम इन Tensors के पार मनमाने ढंग से PyTorch संचालन कर सकते हैं

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
y.get().get()

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
p2p2x.get().get()

In [ ]:
bob._objects

In [ ]:
alice._objects

# अनुभाग 3.2 - Pointer श्रृंखला संचालन

इसलिए अंतिम खंड में जब भी हम एक .send() या .get() ऑपरेशन कहते हैं, तो यह उस ऑपरेशन को सीधे हमारे स्थानीय मशीन पर Tensor पर कॉल करता है। हालाँकि, यदि आपके पास Pointers की एक श्रृंखला है, तो कभी-कभी आप चेन में अंतिम पॉइंटर पर .get () या .send () जैसे ऑपरेशन को कॉल करना चाहते हैं (जैसे कि एक कार्यकर्ता से दूसरे में डेटा सीधे भेजना)। इसे पूरा करने के लिए, आप उन फ़ंक्शन का उपयोग करना चाहते हैं जो विशेष रूप से इस गोपनीयता संरक्षण ऑपरेशन के लिए डिज़ाइन किए गए हैं।

ये ऑपरेशन हैं:

- `my_pointer2pointer.move(another_worker)`

In [ ]:
# x is now a pointer to a pointer to the data which lives on Bob's machine
x = torch.tensor([1,2,3,4,5]).send(bob)

In [ ]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

In [ ]:
x = x.move(alice)

In [ ]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

In [ ]:
x

अति उत्कृष्ट! अब हम एक विश्वसनीय एग्रीगेटर (trusted aggregator) का उपयोग करके रिमोट ग्रेडिएंट औसत (remote gradient averaging) प्रदर्शन करने के लिए उपकरणों से लैस हैं!

# बधाई हो!!! - समुदाय में शामिल होने का समय!

इस नोटबुक ट्यूटोरियल को पूरा करने पर बधाई! यदि आपने इसका आनंद लिया है और एआई और एआई आपूर्ति श्रृंखला (डेटा) के विकेन्द्रीकृत स्वामित्व के संरक्षण की ओर आंदोलन में शामिल होना चाहते हैं, तो आप निम्न तरीकों से ऐसा कर सकते हैं!

### Pysyft को Github पर Star करें!

हमारे समुदाय की मदद करने का सबसे आसान तरीका GitHub repos अभिनीत है! यह हमारे द्वारा बनाए जा रहे कूल टूल्स के बारे में जागरूकता बढ़ाने में मदद करता है।

- [Star PySyft](https://github.com/OpenMined/PySyft)

### हमारी Slack में शामिल हों!

नवीनतम प्रगति पर अद्यतित रहने का सबसे अच्छा तरीका हमारे समुदाय में शामिल होना है! आप फॉर्म भरकर ऐसा कर सकते हैं। [http://slack.openmined.org](http://slack.openmined.org)

### एक कोड प्रोजेक्ट में शामिल हों!

हमारे समुदाय में योगदान करने का सबसे अच्छा तरीका एक कोड योगदानकर्ता बनना है! किसी भी समय आप PySyft GitHub Issues जारी करने वाले पृष्ठ पर जा सकते हैं और "प्रोजेक्ट्स" के लिए फ़िल्टर कर सकते हैं। यह आपको सभी शीर्ष स्तर के टिकट दिखाएगा कि आप किन परियोजनाओं में शामिल हो सकते हैं! यदि आप किसी परियोजना में शामिल नहीं होना चाहते हैं, लेकिन आप थोड़ी सी कोडिंग करना चाहते हैं, तो आप "अच्छा पहला अंक" चिह्नित गीथहब मुद्दों की खोज करके अधिक "वन ऑफ़" मिनी-प्रोजेक्ट्स की तलाश कर सकते हैं।

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### दान करना

यदि आपके पास हमारे कोडबेस में योगदान करने का समय नहीं है, लेकिन फिर भी समर्थन उधार देना चाहते हैं, तो आप हमारे ओपन कलेक्टिव में भी एक बैकर बन सकते हैं। सभी दान हमारी वेब होस्टिंग और अन्य सामुदायिक खर्चों जैसे कि हैकाथॉन और मीटअप की ओर जाते हैं!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)